In [1]:
import pandas as pd
import json
import random
import matplotlib.pyplot as plt
import gc
import copy
import os

In [53]:
data1 = pd.read_csv("task1.csv")
data2 = pd.read_csv("task2.csv")
data = pd.concat([data1,data2],axis = 0)
data = data[[i for i in data if i not in ['example_id','query_id']]]
data = data[data['query_locale'] == 'us']
data = data.drop_duplicates(["query","product_id"])
data = data[[i for i in data if i != "query_locale"]]
data

,query,product_id,esci_label
0,# 2 pencils not sharpened,B0000AQO0O,exact
1,# 2 pencils not sharpened,B0002LCZV4,exact
2,# 2 pencils not sharpened,B00125Q75Y,exact
3,# 2 pencils not sharpened,B001AZ1D3C,exact
4,# 2 pencils not sharpened,B001B097KC,exact
...,...,...,...
1522302,white adirondack chairs,B09C286CSR,exact
1522303,white adirondack chairs,B09CGNKGKQ,exact
1522304,white adirondack chairs,B09CKGY3FW,substitute
1522305,white adirondack chairs,B09D745NK3,exact


In [54]:
# select >5
item_count_df = data_shuffle['product_id'].value_counts().reset_index(drop = False)
item_count_df.columns = ['product_id',"counts"]
item_count_df = item_count_df[item_count_df['counts'] >5]
all_dup_item = set(list(item_count_df['product_id']))

In [55]:
data_shuffle = data.sample(frac = 1).reset_index(drop = True)
print(data_shuffle.shape)
data_shuffle = data_shuffle[data_shuffle['product_id'].isin(all_dup_item)].reset_index(drop = True)
print(data_shuffle.shape)
data_shuffle = data_shuffle.sort_values("product_id").reset_index(drop = True)
data_shuffle

(1399056, 3)
(155092, 3)


,query,product_id,esci_label
0,best books for kids,0060256656,exact
1,keeko childrens book,0060256656,substitute
2,have you filled a bucket today,0060256656,substitute
3,children's books ages 3-5,0060256656,exact
4,rainbow fish book,0060256656,irrelevant
...,...,...,...
155087,small vanity without mirror,B09FXLGQ1S,substitute
155088,bedroom vanity without mirror,B09FXLGQ1S,exact
155089,makeup table without mirror,B09FXLGQ1S,irrelevant
155090,makeup vanity table without mirror,B09FXLGQ1S,irrelevant


In [67]:
# With the same product_id as the axis, construct query_pair and sample
data_merge = pd.merge(data_shuffle, data_shuffle, on = "product_id", how = "left")
data_merge = data_merge[data_merge['query_x'] != data_merge['query_y']]
data_merge = data_merge.sample(frac = 1)
data_merge = data_merge.reset_index(drop = True)
# According to the transitive relationship, construct the label
def make_rel(row):
    if row['esci_label_x'] == "exact":
        return row['esci_label_y']
    elif row['esci_label_y'] == "exact":
        return row['esci_label_x']
    else:
        return ""
data_merge['pair_rel'] = data_merge.apply(lambda x: make_rel(x), axis = 1)
data_merge = data_merge[data_merge["pair_rel"] != ""]
data_merge = data_merge.sample(frac = 1).reset_index(drop = True)
data_merge

,query_x,product_id,esci_label_x,query_y,esci_label_y,pair_rel
0,multivitamin without vitamin k,B004QQ9LVS,substitute,gummy vitamins without iron,exact,substitute
1,king mattress,B07BV29J5N,exact,firm king mattress,exact,exact
3,adult floaties for swimming pool,B00WIMYVMQ,exact,floating net for tanning,exact,exact
5,pura d'or,B00FH692PQ,exact,shampoo without water,exact,exact
6,large face mask for men,B088CV63D6,exact,resuable face mask for virus protection,exact,exact
...,...,...,...,...,...,...
1774257,gluten free atkins,B0015068PA,exact,meal replacement shakes without soy,exact,exact
1774258,mini projetor,B07J5828NS,exact,prokector,exact,exact
1774259,18x18 pillow cover,B07PF35K8Q,exact,throw pillows,exact,exact
1774260,iphone 11 back protector,B07FP41MC5,irrelevant,iphone 11 xs unlocked,exact,irrelevant


In [79]:
data_final = copy.deepcopy(data_merge[['query_x','query_y','pair_rel']])
data_final['pair_rel'][data_final['pair_rel'] == "exact"] = "narrowing"

# A dataframe has two columns "title1" and "title2"
# This situation may occur
# A row: title1 corresponds to "apple", title2 corresponds to "banana"
# Another
# A row: title1 corresponds to "banana", title2 corresponds to "apple"
# We think these two rows are duplicates, how to remove duplicates in this case?
data_final['sorted_title'] = data_final.apply(lambda row: '_'.join(sorted([row['query_x'], row['query_y']])), axis=1)
data_final.drop_duplicates(subset='sorted_title', keep='first', inplace=True)
# remove new 
data_final.drop(columns=['sorted_title'], inplace=True)
data_final

,query_x,query_y,pair_rel
0,fire tablets,all new fire hd 10 tablet,complement
1,10 exercise ball,ball,narrowing
2,usb c to hdmi,macbook pro adapters,narrowing
3,led projector,mini projectors,narrowing
4,1 way window film,garage spring ruler,irrelevant
...,...,...,...
1344843,dlink camera,ring wifi outdoor security camera,substitute
1345871,15gal water container,tap collapsible water bottle,substitute
1347091,xbox one battery chager,xbox one accessories,narrowing
1347692,handheld rainfall shower head with arm,the misugi - 3 mode high pressure shower bath ...,substitute


In [83]:
data_final = data_final.sample(frac = 1)
data_final.to_json(f"pair_data_org.json", orient="records", lines=True)
data_final.to_csv(f"pair_data_org.csv",index = None)

### instruction

In [2]:
pair_rel_data = pd.read_csv("pair_data_org.csv")

In [24]:
all_option_list = [[['narrowing',0],['substitute',0],['complement',0],['irrelevant',0]] for _ in range(len(pair_rel_data))]
all_label_list = []
rel_idx_dict = {'narrowing':0,'substitute':1,'complement':2,'irrelevant':3}
for idx in range(len(pair_rel_data)):
    all_option_list[idx][rel_idx_dict[pair_rel_data['pair_rel'][idx]]][1] = 1
    random.shuffle(all_option_list[idx])
    for option_idx, option in enumerate(all_option_list[idx]):
        if option[1] == 1:
            all_label_list.append(option_idx)
    all_option_list[idx] = "\n".join([f"{option_idx}. {i[0]}" for option_idx,i in enumerate(all_option_list[idx])])

pair_rel_data_copy = copy.deepcopy(pair_rel_data)
pair_rel_data_copy['option'] = all_option_list
pair_rel_data_copy['label'] = all_label_list

In [27]:
prompt_template = "Which of the following statements best describes the relation from query \"{}\" to query \"{}\"?\n{}\nAnswer: "
instruction_data = pd.DataFrame()
instruction_data['prompt'] = pair_rel_data_copy.apply(lambda row:prompt_template.format(row['query_x'],row['query_y'],row['option']),axis = 1)
instruction_data['label'] = pair_rel_data_copy['label']

In [29]:
instruction_data.to_csv("pair_rel_task.csv",index = None)